In [ ]:
from asgardpy.analysis import AsgardpyAnalysis
from asgardpy.config import AsgardpyConfig

from gammapy.modeling.models import Models

import xmltodict
import astropy.units as u
import logging

In [ ]:
from asgardpy.data.dataset_3d import Dataset3DGeneration
from asgardpy.data.target import (
    set_models, create_gal_diffuse_skymodel, 
    create_source_skymodel, read_models_from_asgardpy_config
)

In [ ]:
log = logging.getLogger("test Models from XML")

In [ ]:
config_file = "/path/to/Config.yaml"

In [ ]:
config = AsgardpyConfig()

In [ ]:
%%time
config_main = config.read(config_file)

In [ ]:
for g in config_main.general:
    print(g)

# Steps mentioned in the main config file

In [ ]:
config_main.general.steps

# Target source information

In [ ]:
for c in config_main.target:
    print(c)

# 3D Datasets informations

In [ ]:
for cc in config_main.dataset3d:
    print(cc)

In [ ]:
%%time
analysis = AsgardpyAnalysis(config_main)

In [ ]:
analysis

# Get XML file from a single 3D Dataset - Fermi-LAT

In [ ]:
instruments_list = config_main.dataset3d.instruments
print(len(instruments_list), "number of 3D dataset information provided")

config_3d_dataset = instruments_list[0]
print(f"Instrument selected is {config_3d_dataset.name}")

key_names = config_3d_dataset.dataset_info.key
print(f"The list of different keys or modes of observations for the selected instrument are {key_names}")

In [ ]:
%%time
generate_3d_dataset = Dataset3DGeneration(
    log, config_3d_dataset, config_main
)

In [ ]:
%%time
file_list = generate_3d_dataset.read_to_objects(
    generate_3d_dataset.config_target.components.spectral, 
    key_names[0]
)

In [ ]:
print(f"The XML file selected is {file_list['xml_file']}")
xml_file = file_list['xml_file']

# Read into Models from XML file

In [ ]:
with open(xml_file) as f:
    data = xmltodict.parse(f.read())["source_library"]["source"]

In [ ]:

print("The sources for which model information is givne in the XML file:")
for source in data:
    print(source["@name"])

In [ ]:
print("Diffuse Isotropic Model already initialized")
print(generate_3d_dataset.diffuse_models["diff_iso"])

In [ ]:
diff_gal = create_gal_diffuse_skymodel(generate_3d_dataset.diffuse_models["diff_gal"])
print(diff_gal)

In [ ]:
lp_is_intrinsic = generate_3d_dataset.config_target.components.spectral.model_name == "LogParabola"
aux_path = generate_3d_dataset.config_3d_dataset.io[1].input_dir

print(lp_is_intrinsic, aux_path)

In [ ]:
list_sources = []

for source in data:
    source_name = source["@name"]
    if source_name not in ["IsoDiffModel", "GalDiffModel"]:
        source, is_target_source = create_source_skymodel(
            config_main.target, source, aux_path, lp_is_intrinsic
        )

        if is_target_source:
            list_sources.insert(0, source)
        else:
            list_sources.append(source)

In [ ]:
list_sources.append(generate_3d_dataset.diffuse_models["diff_iso"])

In [ ]:
print(len(list_sources))

In [ ]:
for m in list_sources:
    print(m)

# Reading models from Config file

In [ ]:
spectral_model, spatial_model = read_models_from_asgardpy_config(config_main.target)

In [ ]:
print(spectral_model)

In [ ]:
print(spatial_model)